# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iwanai,42.9744,140.5089,27.35,74,54,2.27,JP,1722637982
1,1,waitangi,-43.9535,-176.5597,8.79,73,49,5.36,NZ,1722637983
2,2,hamilton,39.1834,-84.5333,27.82,76,40,3.58,US,1722637984
3,3,crucecita,-28.9500,-58.9833,30.29,30,0,8.16,AR,1722637985
4,4,iqaluit,63.7506,-68.5145,7.85,57,75,0.00,CA,1722637986


In [25]:
city_data_df[(city_data_df.Country == 'US')].shape


(57, 10)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    hover_cols = ['City','Country']    
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
us_cities_df = city_data_df[city_data_df.Country =='US']

# Drop any rows with null values
us_cities_df.dropna()

# Display sample data
us_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,hamilton,39.1834,-84.5333,27.82,76,40,3.58,US,1722637984
5,5,lompoc,34.6391,-120.4579,28.10,46,0,4.90,US,1722637988
15,15,ketchikan,55.3422,-131.6461,21.86,56,0,8.23,US,1722637998
17,17,coos bay,43.3665,-124.2179,23.52,73,75,5.66,US,1722638000
42,42,kailua-kona,19.6406,-155.9956,28.98,74,40,5.66,US,1722638028


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = us_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hamilton,39.1834,-84.5333,27.82,76,40,3.58,US,1722637984,
5,5,lompoc,34.6391,-120.4579,28.10,46,0,4.90,US,1722637988,
15,15,ketchikan,55.3422,-131.6461,21.86,56,0,8.23,US,1722637998,
17,17,coos bay,43.3665,-124.2179,23.52,73,75,5.66,US,1722638000,
42,42,kailua-kona,19.6406,-155.9956,28.98,74,40,5.66,US,1722638028,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params = params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
hamilton - nearest hotel: North Vista Manor
lompoc - nearest hotel: Red Roof Inn Lompoc
ketchikan - nearest hotel: Cape Fox Lodge
coos bay - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
bethel - nearest hotel: Hampton Inn Danbury
washington - nearest hotel: Cashmere Inn
kodiak - nearest hotel: No hotel found
clarksville - nearest hotel: Riverview Inn
freeport - nearest hotel: The Freeport Inn and Marina
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
brookings - nearest hotel: Quality Inn
hawaiian paradise park - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
albany - nearest hotel: No hotel found
big spring - nearest hotel: Settles Hotel
san patricio - nearest hotel: No hotel found
red hill - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
lander - nearest hotel: Holiday Lodge
westport - nearest hotel: Norwalk Inn and Conference

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hamilton,39.1834,-84.5333,27.82,76,40,3.58,US,1722637984,North Vista Manor
5,5,lompoc,34.6391,-120.4579,28.10,46,0,4.90,US,1722637988,Red Roof Inn Lompoc
15,15,ketchikan,55.3422,-131.6461,21.86,56,0,8.23,US,1722637998,Cape Fox Lodge
17,17,coos bay,43.3665,-124.2179,23.52,73,75,5.66,US,1722638000,No hotel found
42,42,kailua-kona,19.6406,-155.9956,28.98,74,40,5.66,US,1722638028,Kona Seaside Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name','Country']    
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)